In [30]:
import tapas.datasets
import tapas.generators
import tapas.threat_models
import tapas.attacks
import tapas.report

In [31]:
data = tapas.datasets.TabularDataset.read("data/microdata-2011")

/home/flavio/repositories/projects/GANS/explore-attacks/.venv/lib/python3.9/site-packages/tapas/datasets/dataset.py:37: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(fp, header=validate_header(fp, cnames), dtype=dtypes, index_col=None, names=cnames)


Notes
- there is no continuous variable in this census data set. does this matter for some generators?

In [32]:
display(type(data))
dir(data)
type(data.data)
data.data.head()


tapas.datasets.dataset.TabularDataset

(569743, 17)

the first 2 rows are odd??

In [22]:
generator = tapas.generators.Raw()

it's not clear to me how the generator is used downstream. ie, in this quickstart example, we only define the generator, and then pass it on to the threat model (through `sdg_knowledge`). 
- this must imply that generator.train() is called somewhere in the attack?
    - The combination of all inputs to threat_model make all parts work together 
    - Thus, it is crucial to define the data type properly when using our own data/generator (following the instructions above)

Below we see how to use `generator.generate()`.

In [23]:
generator.fit(data.data)
generator.generate(3)

,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,Approximated Social Grade
7623679,E12000002,H,1,1,2,8,5,2,1,3,1,2,5,6,11,-9,4
7942794,W92000004,H,1,1,1,4,4,2,1,2,1,1,1,2,2,3,1
7893106,E12000007,H,3,1,1,2,1,2,1,3,1,6,3,-9,-9,-9,4


In [24]:
dir(generator)
generator.generate??
tapas.generators.Raw??

Init signature: tapas.generators.Raw()
Source:        
class Raw(Generator):
    """This generator simply samples from the real data."""
    def __init__(self):
        super().__init__()

    def fit(self, dataset):
        self.dataset = dataset
        self.trained = True

    def generate(self, num_samples = None, random_state = None):
        if self.trained: 
            if num_samples is None:
                return self.dataset
            return self.dataset.sample(num_samples, random_state = random_state)
        else:
            raise RuntimeError("No dataset provided to generator")
        
    def __call__(self, dataset, num_samples, random_state = None):
        self.fit(dataset)
        return self.generate(num_samples, random_state = random_state)

    @property
    def label(self):
        return "Raw"
File:           ~/repositories/projects/GANS/explore-attacks/.venv/lib/python3.9/site-packages/tapas/generators/generator.py
Type:           ABCMeta
Subclasses:     

Define attacker's knowledge about the data set 

In [25]:
data_knowledge = tapas.threat_models.AuxiliaryDataKnowledge(
      data,
      auxiliary_split=0.5,
      num_training_records=5000
)

Define knowledge about the generator

In [26]:
sdg_knowledge = tapas.threat_models.BlackBoxKnowledge(
    generator,
    num_synthetic_records=5000,
)

Define goal of the attacker: membership inference attack

In [27]:
threat_model = tapas.threat_models.TargetedMIA(
   attacker_knowledge_data=data_knowledge,
   target_record=data.get_records([0]),
   attacker_knowledge_generator=sdg_knowledge,
   generate_pairs=True, # see API. SOURCE?
   replace_target=True
)

#### Attack

In [28]:
attacker = tapas.attacks.GroundhogAttack()

This is equivalent to a particular instantiation of a larger class of attacks. The current code on the website does not work though.

In [29]:
attacker.train(threat_model, num_samples=1000)

ValueError: nan is not in list

Issues
- sklearn? -- but I used a different approach from what they recommend
- instantiating the attack as a special case of a general attack
- `attacker.train()`